# Machine Learning
Ingest the data we have gathered into a neural network. 

In [4]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import feature_column
from sklearn.model_selection import train_test_split

df = pd.read_csv('samples_10k.csv')

df['size_dif'] = df['size_of_image'] - df['file_size']
df.sample(3)

,index,label,file_type,file_size,file_entropy,imports,exports,size_of_image,size_of_code,size_of_stack_reserve,size_of_stack_commit,size_of_heap_reserve,size_of_heap_commit,number_of_sections,sections,size_dif
7944,9299,1,"PE32 executable (GUI) Intel 80386, for MS Windows",898312,7.957193,"['DispatchMessageA', 'TranslateMessage', 'Post...",[],905216,52736,1048576,4096,1048576,4096,6,"[['.text\x00\x00\x00', 52736, 52411, 1, 1, 0],...",6904
8076,242663,0,"PE32 executable (DLL) (console) Intel 80386, f...",270672,6.840944,"['_except_handler4_common', '_initterm', 'wcsc...",[],278528,80896,262144,4096,1048576,4096,6,"[['.text\x00\x00\x00', 80896, 80419, 1, 1, 0],...",7856
8073,68512,1,PE32 executable (GUI) Intel 80386 (stripped to...,59288,7.693866,"['LoadLibraryA', 'GetProcAddress', 'VirtualPro...",[],167936,49152,2097152,4096,1048576,4096,3,"[['UPX0\x00\x00\x00\x00', 0, 102400, 0, 1, 1],...",108648


In [6]:
# Help ensure our datasets are within the same range using ZScore. 
def ZScore(df):
    return (df-df.min())/(df.max()-df.min())

zscore_candidates = [
    'file_size', 'size_of_image', 'size_of_code', 'size_of_stack_reserve',
    'size_of_stack_commit', 'size_of_heap_reserve', 'size_of_heap_commit',
    'size_dif', 'file_entropy', 
]
for candidate in zscore_candidates:
    df[candidate] = ZScore(df[candidate])

df.describe()

,index,label,file_size,file_entropy,size_of_image,size_of_code,size_of_stack_reserve,size_of_stack_commit,size_of_heap_reserve,size_of_heap_commit,number_of_sections,size_dif
count,10217.000000,10217.000000,10217.000000,10217.000000,10217.000000,10217.000000,10217.000000,10217.000000,10217.000000,10217.000000,10217.000000,10217.000000
mean,195419.332974,0.562788,0.006705,6.392952,0.005912,0.001044,0.030081,0.003863,0.248888,0.003618,4.706274,0.157604
std,157455.572825,0.496066,0.017053,1.119250,0.024199,0.029478,0.032580,0.028604,0.072377,0.010106,1.931064,0.017782
min,9.000000,0.000000,0.000000,0.038443,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,51918.000000,0.000000,0.000774,5.915500,0.000599,0.000013,0.031250,0.001953,0.250000,0.003497,3.000000,0.156524
50%,196313.000000,1.000000,0.002615,6.450281,0.001676,0.000043,0.031250,0.001953,0.250000,0.003497,5.000000,0.156587
75%,245513.000000,1.000000,0.006686,7.182155,0.004550,0.000148,0.031250,0.001953,0.250000,0.003497,6.000000,0.156654
max,531492.000000,1.000000,1.000000,7.999642,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,40.000000,1.000000
